In [1]:
import requests
import os
import shutil

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

In [3]:
url_start = "https://www.basketball-reference.com/awards/awards_{}.html"
years = list(range(1991,2022))
for year in years:
    url = url_start.format(year)
    
    data = requests.get(url)
    
    with open("mvp/{}.html".format(year), "w+") as f:
        f.write(data.text)

In [55]:
from bs4 import BeautifulSoup

In [63]:
with open("mvp/1991.html") as f:
    page = f.read()
    


In [65]:
soup = BeautifulSoup(page, 'html.parser')



In [66]:
soup.find('tr', class_="over_header").decompose()

AttributeError: 'NoneType' object has no attribute 'decompose'

In [61]:
import pandas as pd
mvp_table = soup.find_all(id="mvp")
print(mvp_table)

[]


In [ ]:
mvp_1991 = pd.read_html(str(mvp_table))[0]
mvp_1991.head(1)

,Rank,Player,Age,Tm,First,Pts Won,Pts Max,Share,G,MP,PTS,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48
0,1,Giannis Antetokounmpo,25,MIL,85.0,962.0,1010,0.952,63,30.4,29.5,13.6,5.6,1.0,1.0,0.553,0.304,0.633,11.1,0.279


In [ ]:
mvp_1991["Year"] = 1991
mvp_1991.head()

,Rank,Player,Age,Tm,First,Pts Won,Pts Max,Share,G,MP,...,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48,Year
0,1,Michael Jordan,27,CHI,77.0,891.0,960,0.928,82,37.0,...,6.0,5.5,2.7,1.0,0.539,0.312,0.851,20.3,0.321,1991
1,2,Magic Johnson,31,LAL,10.0,497.0,960,0.518,79,37.1,...,7.0,12.5,1.3,0.2,0.477,0.320,0.906,15.4,0.251,1991
2,3,David Robinson,25,SAS,6.0,476.0,960,0.496,82,37.7,...,13.0,2.5,1.5,3.9,0.552,0.143,0.762,17.0,0.264,1991
3,4,Charles Barkley,27,PHI,2.0,222.0,960,0.231,67,37.3,...,10.1,4.2,1.6,0.5,0.570,0.284,0.722,13.4,0.258,1991
4,5,Karl Malone,27,UTA,0.0,142.0,960,0.148,82,40.3,...,11.8,3.3,1.1,1.0,0.527,0.286,0.770,15.5,0.225,1991


In [ ]:
print(mvp_table)


<table class="sortable stats_table" data-cols-to-freeze=",2" id="mvp">
<caption>Most Valuable Player (Michael Jordan Trophy) Table</caption>
<colgroup><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/></colgroup>
<thead>

<tr>
<th aria-label="rank" class="poptip center" data-stat="rank" scope="col">Rank</th>
<th aria-label="Player" class="poptip sort_default_asc left" data-stat="player" scope="col">Player</th>
<th aria-label="Age" class="poptip sort_default_asc center" data-stat="age" data-tip="Player's age on February 1 of the season" scope="col">Age</th>
<th aria-label="Tm" class="poptip sort_default_asc left" data-stat="team_id" data-tip="Team" scope="col">Tm</th>
<th aria-label="First" class="poptip right" data-over-header="Voting" data-stat="votes_first" data-tip="First Place Votes" scope="col">First</th>
<th aria-label="Pts Won" class="poptip right" data-over-header="Voting" data-stat="points_won" scope="col">P